In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

# Queries for each rescue type
water_query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
mountain_query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
disaster_query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}

# Load logo image in base64 format
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),

    html.Div([
        html.H1('SNHU CS-340 Dashboard', style={'margin-right': '20px'}),
        html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '100px'})
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),

    html.Hr(),

    # Filter by rescue type
    html.Label("Filter by Rescue Type"),
    dcc.RadioItems(
        id='rescue-type-selector',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'width': '100px', 'maxWidth': '180px', 'whiteSpace': 'normal'}
    ),

    html.Br(),
    html.Hr(),

    # Side-by-side layout for map and pie chart
    html.Div([
        html.Div(id='map-id', className='col s12 m6', style={'width': '50%'}),
        html.Div(id='pie-chart-id', style={'width': '50%'})
    ], style={'display': 'flex'}),

    html.H3("Abhilash Krishna Raj SNHU CS-340 MongoDB Authentication")
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row, 9])
                                 ])
                             ])
               ])
    ]

# Update table based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-selector', 'value')]
)
def filter_table(selected_rescue_type):
    if selected_rescue_type == 'water':
        data = shelter.read(water_query)
    elif selected_rescue_type == 'mountain':
        data = shelter.read(mountain_query)
    elif selected_rescue_type == 'disaster':
        data = shelter.read(disaster_query)
    else:
        data = shelter.read({})

    df_filtered = pd.DataFrame.from_records(data)
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Add callback for second chart (pie chart)
@app.callback(
    Output('pie-chart-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    dff = pd.DataFrame(data)
    if dff.empty or 'breed' not in dff.columns:
        return html.Div("No data to display.")
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)

app.run_server(debug=True)


Dash app running on http://127.0.0.1:18589/
